Get total value lost of critical materials (SG + CN)

In [1]:

import pandas as pd;

inventory_df = pd.read_csv("data/Inventory.csv")
china_df = pd.read_csv("MaterialMaster_with_clusters_CHINA.csv")
singapore_df = pd.read_csv("MaterialMaster_with_clusters_SINGAPORE.csv")

# Filter for Cluster = 0 (Critical Material)
china_critical = china_df[china_df["Cluster"] == 0]
singapore_critical = singapore_df[singapore_df["Cluster"] == 0]

# Combine critical materials
critical_materials = pd.concat([china_critical, singapore_critical])

# Merge inventory_df with critical materials
merged_df = inventory_df[inventory_df["BALANCE_AS_OF_DATE"] == "12/31/2024"].merge(
    critical_materials[["MATERIAL_NAME", "DOWNGRADE_VALUE_AVG_MT"]],
    on="MATERIAL_NAME",
    how="inner"
)

# Convert unrestricted stock from KG to MT
merged_df["UNRESRICTED_STOCK_MT"] = merged_df["UNRESRICTED_STOCK"] / 1000

# Calculate downgrade value lost
merged_df["DOWNGRADE_VALUE_LOST"] = (
    merged_df["UNRESRICTED_STOCK_MT"] * merged_df["DOWNGRADE_VALUE_AVG_MT"]
)

# Group by material and plant for detailed summary
summary_df = merged_df.groupby(["PLANT_NAME", "MATERIAL_NAME"]).agg({
    "UNRESRICTED_STOCK_MT": "sum",
    "DOWNGRADE_VALUE_LOST": "sum"
}).reset_index()

# Calculate total downgrade value lost
total_downgrade_value_lost = summary_df["DOWNGRADE_VALUE_LOST"].sum()

# Optionally: print or export results
print(summary_df)
print(f"Total Downgrade Value Lost: {total_downgrade_value_lost}")

downgrade_by_plant = merged_df.groupby("PLANT_NAME")["DOWNGRADE_VALUE_LOST"].sum().reset_index()
print(downgrade_by_plant)

             PLANT_NAME MATERIAL_NAME  UNRESRICTED_STOCK_MT  \
0       CHINA-WAREHOUSE      MAT-0008                 0.000   
1       CHINA-WAREHOUSE      MAT-0013                68.800   
2       CHINA-WAREHOUSE      MAT-0017              2014.625   
3       CHINA-WAREHOUSE      MAT-0047               320.975   
4       CHINA-WAREHOUSE      MAT-0082                 3.750   
..                  ...           ...                   ...   
70  SINGAPORE-WAREHOUSE      MAT-0393                23.375   
71  SINGAPORE-WAREHOUSE      MAT-0412                 7.200   
72  SINGAPORE-WAREHOUSE      MAT-0419               301.075   
73  SINGAPORE-WAREHOUSE      MAT-0426              4794.000   
74  SINGAPORE-WAREHOUSE      MAT-0428               264.950   

    DOWNGRADE_VALUE_LOST  
0           0.000000e+00  
1           1.849001e+05  
2           5.734898e+06  
3           8.213830e+05  
4           9.596383e+03  
..                   ...  
70          1.034356e+04  
71          3.499200e+03  
